<a href="https://colab.research.google.com/github/pranitha2144/Internship-iith/blob/Resnet/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

In [42]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [44]:
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [45]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [46]:
from tensorflow.keras.optimizers import Adam

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final softmax layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [54]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/data_liver/train',
                                                    batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size = (224, 224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( '/content/drive/My Drive/data_liver/val',
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (224, 224))

Found 700 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [55]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/content/drive/My Drive/Best models/model_n_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
hist=model.fit_generator(train_generator,
            validation_data = validation_generator,
            epochs = 10,
            verbose = 2)

Epoch 1/10
35/35 - 95s - loss: 0.7373 - acc: 0.7000 - val_loss: 0.8316 - val_acc: 0.6500
Epoch 2/10
35/35 - 94s - loss: 0.6855 - acc: 0.6957 - val_loss: 0.7465 - val_acc: 0.6900
Epoch 3/10
35/35 - 94s - loss: 0.6370 - acc: 0.7300 - val_loss: 0.7289 - val_acc: 0.6600
Epoch 4/10
35/35 - 97s - loss: 0.6264 - acc: 0.7329 - val_loss: 0.8187 - val_acc: 0.6300
Epoch 5/10
35/35 - 94s - loss: 0.6389 - acc: 0.7300 - val_loss: 0.6044 - val_acc: 0.7300
Epoch 6/10
35/35 - 94s - loss: 0.6229 - acc: 0.7329 - val_loss: 0.6928 - val_acc: 0.6900
Epoch 7/10
35/35 - 94s - loss: 0.6404 - acc: 0.7200 - val_loss: 0.7000 - val_acc: 0.6800
Epoch 8/10
35/35 - 95s - loss: 0.5578 - acc: 0.7843 - val_loss: 0.5758 - val_acc: 0.7600
Epoch 9/10
35/35 - 94s - loss: 0.6301 - acc: 0.7143 - val_loss: 0.5846 - val_acc: 0.7500
Epoch 10/10
35/35 - 96s - loss: 0.6602 - acc: 0.7357 - val_loss: 0.6295 - val_acc: 0.7100


In [56]:
last_layer = pre_trained_model.get_layer('mixed7') #Only use layers including and above 'mixed7'
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [57]:
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            epochs = 10,
            verbose = 2)

last layer output shape:  (None, 12, 12, 768)
Epoch 1/10
35/35 - 95s - loss: 1.4645 - acc: 0.5143 - val_loss: 0.9011 - val_acc: 0.6600
Epoch 2/10
35/35 - 95s - loss: 0.7889 - acc: 0.6686 - val_loss: 0.8084 - val_acc: 0.6700
Epoch 3/10
35/35 - 94s - loss: 0.6670 - acc: 0.6886 - val_loss: 0.6364 - val_acc: 0.7400
Epoch 4/10
35/35 - 94s - loss: 0.5926 - acc: 0.7500 - val_loss: 0.5494 - val_acc: 0.7700
Epoch 5/10
35/35 - 93s - loss: 0.5520 - acc: 0.7743 - val_loss: 0.7286 - val_acc: 0.6800
Epoch 6/10
35/35 - 93s - loss: 0.5308 - acc: 0.8000 - val_loss: 0.5292 - val_acc: 0.7700
Epoch 7/10
35/35 - 93s - loss: 0.5201 - acc: 0.7814 - val_loss: 0.5629 - val_acc: 0.7700
Epoch 8/10
35/35 - 95s - loss: 0.4837 - acc: 0.7886 - val_loss: 0.5398 - val_acc: 0.7800
Epoch 9/10
35/35 - 93s - loss: 0.4960 - acc: 0.7900 - val_loss: 0.5571 - val_acc: 0.7400
Epoch 10/10
35/35 - 94s - loss: 0.5062 - acc: 0.7900 - val_loss: 0.5073 - val_acc: 0.7700
